## Import the required packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import numpy as np

## I. Data Cleaning

#### Load the raw CSV file

In [2]:
# Verify your working directory
print("Current working directory:", os.getcwd())

# Select your raw CSV for taxi trip records
Tk().withdraw()  # Hide the root window
file_path = askopenfilename(title="Select your local taxi data CSV (ensure it's the raw data)")
original_filename = os.path.basename(file_path) 

# Check and load
if not file_path or not os.path.exists(file_path):
    raise FileNotFoundError("File not found or not selected.")
else:
    df = pd.read_csv(file_path)
    print("Loaded file:", file_path)
    print("Initial shape:", df.shape)

Current working directory: c:\diksha\Summer Sem\DataAnalysis\Notebooks\Cleaning
Loaded file: C:/diksha/Summer Sem/DataAnalysis/Data/raw data/2022_Dec_Taxi.csv
Initial shape: (3344233, 19)


#### Initial Inspection

In [3]:
print("\nData Info:")
df.info()


Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3344233 entries, 0 to 3344232
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
 18  airport_fee            float64
dtypes: float64(12), int64(4), object(3)
memory usage: 484.8+ MB


In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
# Print the first few rows for the dataframe
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,12/01/2022 12:00:00 AM,12/01/2022 12:09:48 AM,1.0,1.11,1.0,N,230,161,1,7.50,0.5,0.5,2.26,0.00,0.3,13.56,2.5,0.00
1,2,12/01/2022 12:00:00 AM,12/01/2022 12:17:00 AM,NaN,6.80,NaN,NaN,87,262,0,31.15,0.0,0.5,5.17,0.00,0.3,39.62,NaN,NaN
2,2,12/01/2022 12:00:01 AM,12/01/2022 12:28:59 AM,2.0,17.46,2.0,N,132,48,2,52.00,0.0,0.5,0.00,6.55,0.3,63.10,2.5,1.25
3,2,12/01/2022 12:00:03 AM,12/01/2022 12:31:34 AM,1.0,6.65,1.0,N,48,97,1,25.00,0.5,0.5,5.76,0.00,0.3,34.56,2.5,0.00
4,2,12/01/2022 12:00:03 AM,12/01/2022 12:14:33 AM,1.0,8.81,1.0,N,138,140,1,25.00,0.5,0.5,0.01,6.55,0.3,36.61,2.5,1.25


#### Standardized Timestamps

In [6]:
# Convert datetime columns 
# Current Format '01/01/2023 12:00:00 AM'
# New Format: '%m/%d/%Y %I:%M:%S %p'
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p',errors='coerce')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%m/%d/%Y %I:%M:%S %p',errors='coerce')

In [7]:
# Localize timestamps to NYC timezone
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].dt.tz_localize(
    'America/New_York',
    ambiguous='NaT',
    nonexistent='shift_forward'
)

df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].dt.tz_localize(
    'America/New_York',
    ambiguous='NaT',
    nonexistent='shift_forward'
)

In [8]:
# Confirm that timestamps are now timezone-aware
print("Timezone info (pickup):", df['tpep_pickup_datetime'].dt.tz)

Timezone info (pickup): America/New_York


#### Trip Duration Calculation

In [9]:
# Compute trip duration (in minutes)
df['trip_duration_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [10]:
# Drop rows with NaT or negative durations
df = df[df['trip_duration_min'].notna()]
df = df[df['trip_duration_min'] > 0]

#### Data Type Conversion 

In [11]:
# Numeric conversion
num_cols = ['trip_distance', 'fare_amount', 'tip_amount', 'total_amount']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# Categorical conversion
cat_cols = ['RatecodeID', 'payment_type', 'VendorID']
df[cat_cols] = df[cat_cols].astype('category')

#### Filter Out Invalid/Extreme Values

In [12]:
df = df[(df['trip_distance'] > 0) & (df['trip_distance'] < 100)] # Trip distance - Min: 0 miles, Max: 100 miles
df = df[df['fare_amount'] > 0] # Tripe fare amount must be greater than 0
df = df[df['trip_duration_min'] < 240]  # Trip duration is under 4 hours

#### Add Date and Time Features

In [13]:
# add pickup date and time features 
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek  # 0 = Monday

# add pickup date and time features 
df['droppoff_date'] = df['tpep_pickup_datetime'].dt.date
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek  # 0 = Monday

#### Check Pickup and Dropoff Location IDs

In [14]:
# Filter invalid PULocationID and DOLocationID
initial_rows = df.shape[0]
initial_rows
df = df[(df['PULocationID'].between(1, 263)) & (df['DOLocationID'].between(1, 263))]
print(f"Shape after location ID filtering: {df.shape}")

Shape after location ID filtering: (3189384, 24)


#### Load Taxi Zone Lookup File

In [15]:
Tk().withdraw()  # Hide the root window
zone_file_path = askopenfilename(title="Select the taxi zone lookup CSV")

In [16]:
# Check and load
if not zone_file_path or not os.path.exists(zone_file_path):
    raise FileNotFoundError("Zone lookup file not found or not selected.")
else:
    zones = pd.read_csv(zone_file_path, keep_default_na=True, delimiter=',', skipinitialspace=True)
    zone_lookup = zones.copy()
    print("Loaded zone lookup file:", zone_file_path)
    print("Zone file shape:", zones.shape)

Loaded zone lookup file: C:/diksha/Summer Sem/DataAnalysis/Data/raw data/taxi_zone_lookup.csv
Zone file shape: (265, 4)


In [17]:
# Merge pickup location info
df = df.merge(
    zone_lookup.rename(columns={
        "LocationID": "PULocationID",
        "Zone": "pickup_zone",
        "Borough": "pickup_borough",
        "service_zone": "pickup_service_zone"
    }),
    on="PULocationID", how="left"
)

In [18]:
# Merge dropoff location info
df = df.merge(
    zone_lookup.rename(columns={
        "LocationID": "DOLocationID",
        "Zone": "dropoff_zone",
        "Borough": "dropoff_borough",
    }),
    on="DOLocationID", how="left"
)

#### Drop Columns

In [19]:
columns_to_drop = [
    'VendorID', 'RatecodeID', 'store_and_fwd_flag', 'passenger_count',
    'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
    'improvement_surcharge', 'congestion_surcharge', 'airport_fee',
    'total_amount',  # derived field
    'PULocationID', 'DOLocationID'  # drop since we now have zone/borough
]
df = df.drop(columns=columns_to_drop)

#### Add Additional Features

In [20]:
# Remove invalid or noisy trips first
df = df[df['trip_duration_min'] >= 1]         # Avoid 0- or short-duration trips
df = df[df['fare_amount'] >= 1]               # Filter refunded or nonsensical fares
df = df[df['trip_distance'] > 0]              # Prevent division by zero in fare_per_mile


In [21]:
# Derived feature for fare per minute (target variable)
df['fare_per_minute'] = df['fare_amount'] / df['trip_duration_min']

# Filter out extreme fare per minute values
# This removes any fare_per_minute that is zero or above the 99th percentile
df = df[(df['fare_per_minute'] > 0) & (df['fare_per_minute'] < df['fare_per_minute'].quantile(0.99))]


In [22]:
# Cap extremely high trip distances (top 1%) to reduce fare_per_mile distortion
max_distance = df['trip_distance'].quantile(0.99)
df = df[df['trip_distance'] <= max_distance]


In [23]:
# Derived feature for fare per mile (feature important for distance-based pricing)
# Compute fare per mile
df['fare_per_mile'] = df['fare_amount'] / df['trip_distance']
df['fare_per_mile'] = df['fare_per_mile'].replace([np.inf, -np.inf], np.nan)
df = df[df['fare_per_mile'].notna()]


In [24]:
# Derived feature for trip speed
df['trip_speed'] = df['trip_distance'] / df['trip_duration_min']  # miles per minute
df['trip_speed_mph'] = df['trip_speed'] * 60
df = df[(df['trip_speed_mph'] > 1) & (df['trip_speed_mph'] < 60)] # reasonable speed limits


In [25]:
# Deriving feature for time of day based on segmenting
def time_of_day(hour):
    if 0 <= hour < 5:
        return 'Early Morning'
    elif 5 <= hour < 10:
        return 'Morning Rush'
    elif 10 <= hour < 15:
        return 'Midday'
    elif 15 <= hour < 19:
        return 'Evening Rush'
    else:
        return 'Night'
df['time_of_day'] = df['pickup_hour'].apply(time_of_day)

In [26]:
# Create weekday/weekend label
df['day_type'] = df['pickup_day_of_week'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')
#Add Day Type (Weekday vs Weekend) weekend flag
df['is_weekend'] = df['pickup_day_of_week'].isin([5, 6])  # Saturday = 5, Sunday = 6

In [27]:
# Define known airport zones 
airport_keywords = [
    'JFK', 'John F Kennedy', 'Kennedy',
    'LGA', 'LaGuardia',
    'EWR', 'Newark'
]

# Convert to string and check for substring match
df['is_airport_pickup'] = df['pickup_zone'].astype(str).str.contains('|'.join(airport_keywords), case=False, na=False)
df['is_airport_dropoff'] = df['dropoff_zone'].astype(str).str.contains('|'.join(airport_keywords), case=False, na=False)

# Flag for any trip involving an airport
df['is_airport_trip'] = df['is_airport_pickup'] | df['is_airport_dropoff']

In [28]:
print("Total airport trips:", df['is_airport_trip'].sum())
print("Pickup at airport only:", df['is_airport_pickup'].sum())
print("Dropoff at airport only:", df['is_airport_dropoff'].sum())


Total airport trips: 297569
Pickup at airport only: 217114
Dropoff at airport only: 86030


#### Save Clean DF to CSV

In [29]:
df['fare_amount'].describe()

count    3.112536e+06
mean     1.592659e+01
std      1.298456e+01
min      1.060000e+00
25%      7.900000e+00
50%      1.150000e+01
75%      1.850000e+01
max      2.500000e+02
Name: fare_amount, dtype: float64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3112536 entries, 0 to 3189383
Data columns (total 25 columns):
 #   Column                 Dtype                           
---  ------                 -----                           
 0   tpep_pickup_datetime   datetime64[ns, America/New_York]
 1   tpep_dropoff_datetime  datetime64[ns, America/New_York]
 2   trip_distance          float64                         
 3   fare_amount            float64                         
 4   trip_duration_min      float64                         
 5   pickup_date            object                          
 6   pickup_hour            int32                           
 7   pickup_day_of_week     int32                           
 8   droppoff_date          object                          
 9   pickup_borough         object                          
 10  pickup_zone            object                          
 11  pickup_service_zone    object                          
 12  dropoff_borough        object    

#### Convert Data Types Again

In [31]:
cat_cols = ['pickup_borough', 'pickup_zone', 
            'dropoff_borough', 'dropoff_zone',
            'time_of_day', 'day_type', ]

for col in cat_cols:
    df[col] = df[col].astype("category")

In [32]:
df['pickup_date'] = pd.to_datetime(df['pickup_date'])
df['droppoff_date'] = pd.to_datetime(df['droppoff_date'])

### Enforce Consistant Data Types (used for scaling)

In [33]:
numeric_cols = [
    'trip_distance', 'fare_amount', 'trip_duration_min',
    'fare_per_minute', 'trip_speed', 'trip_speed_mph'
]

int_cols = ['pickup_hour', 'pickup_day_of_week']
bool_cols = ['is_weekend']

# Convert numeric fields to float
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)

# Convert integer fields to int
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(int)

# Ensure boolean field is properly typed
df['is_weekend'] = df['is_weekend'].astype(bool)


print(" Data types standardized. Ready for export or modeling.")

 Data types standardized. Ready for export or modeling.


In [34]:
print(df.shape)  # Final row count should be > 0
print(df['tpep_pickup_datetime'].notna().sum())  # Should match row count


(3112536, 25)
3112536


#### Save Cleaned Dataframe to CSV

In [35]:
default_filename = "Clean_" + original_filename

# Hide the root Tkinter window
Tk().withdraw()

# Open save file dialog (no default path logic)
save_path = asksaveasfilename(
    initialfile=default_filename,
    title="Select where to save cleaned taxi data CSV",
    defaultextension=".csv",
    filetypes=[("CSV files", "*.csv")]
)

if save_path:
    df.to_csv(save_path, index=False)
    print(f"Cleaned dataset saved to: {save_path}")
    print(f"Final shape: {df.shape}")
else:
    print("Save cancelled, file was not saved.")

Cleaned dataset saved to: C:/diksha/Summer Sem/ScoringModel/Data/cleaned/Clean_2022_Dec_Taxi.csv
Final shape: (3112536, 25)


In [36]:
df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance',
       'fare_amount', 'trip_duration_min', 'pickup_date', 'pickup_hour',
       'pickup_day_of_week', 'droppoff_date', 'pickup_borough', 'pickup_zone',
       'pickup_service_zone', 'dropoff_borough', 'dropoff_zone',
       'service_zone', 'fare_per_minute', 'fare_per_mile', 'trip_speed',
       'trip_speed_mph', 'time_of_day', 'day_type', 'is_weekend',
       'is_airport_pickup', 'is_airport_dropoff', 'is_airport_trip'],
      dtype='object')